In [10]:
# An example of using multiprocessing to write to an Icechunk dataset
import multiprocessing as mp
import glob
import time
import itertools
import os
import shutil
import tempfile
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, process

import numpy as np
import pandas as pd
import xarray as xr
from icechunk.xarray import to_icechunk
from icechunk import Repository, Session, local_filesystem_storage, s3_storage

In [13]:
storage_config = s3_storage(
    bucket="icechunk",
    prefix="giops_day",
    access_key_id="admin",
    secret_access_key="password",
    endpoint_url="http://142.130.249.35:9000",
    allow_http=True,
    force_path_style=True,
)

repo = Repository.open(storage_config)

In [14]:
session = repo.readonly_session("main")

In [15]:
ds = xr.open_zarr(session.store,consolidated=False)
ds

<xarray.Dataset> Size: 1GB
Dimensions:             (time: 1, variable: 1, depth: 50, latitude: 850,
                         longitude: 1800)
Coordinates:
  * depth               (depth) float32 200B 0.494 1.541 ... 5.275e+03 5.728e+03
  * longitude           (longitude) float32 7kB 0.0 0.2 0.4 ... 359.6 359.8
  * latitude            (latitude) float32 3kB -80.0 -79.8 -79.6 ... 89.6 89.8
  * variable            (variable) <U8 32B 'vozocrtx'
  * time                (time) datetime64[ns] 8B 2025-09-09
Data variables:
    forecast_timestamp  (time, variable) int64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    model_run           (time, variable) int64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    votemper            (time, depth, latitude, longitude) float32 306MB dask.array<chunksize=(1, 7, 107, 450), meta=np.ndarray>
    vosaline            (time, depth, latitude, longitude) float32 306MB dask.array<chunksize=(1, 7, 107, 450), meta=np.ndarray>
    vozocrtx            (time, depth, latitude, longitude) float32 306MB dask.array<chunksize=(1, 7, 107, 450), meta=np.ndarray>
    vomecrty            (time, depth, latitude, longitude) float32 306MB dask.array<chunksize=(1, 7, 107, 450), meta=np.ndarray>
Attributes:
    Conventions:      CF-1.6
    title:            Ice ocean analysis fields
    institution:      The Canadian Centre for Meteorological and Environmenta...
    source:           Global Ice Ocean Prediction System: version 3.4.0
    product_version:  GIOPS_3.4.0
    creation_date:    2025-Sep-09 02:18:19 UTC
    contact:          production-info@ec.gc.ca

In [16]:
ds.time

<xarray.DataArray 'time' (time: 1)> Size: 8B
array(['2025-09-09T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 8B 2025-09-09
Attributes:
    long_name:      Validity time
    standard_name:  time
    axis:           T

In [17]:
ds.vomecrty[0,:,:,:]

<xarray.DataArray 'vomecrty' (depth: 50, latitude: 850, longitude: 1800)> Size: 306MB
dask.array<getitem, shape=(50, 850, 1800), dtype=float32, chunksize=(7, 107, 450), chunktype=numpy.ndarray>
Coordinates:
  * depth      (depth) float32 200B 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * longitude  (longitude) float32 7kB 0.0 0.2 0.4 0.6 ... 359.4 359.6 359.8
  * latitude   (latitude) float32 3kB -80.0 -79.8 -79.6 -79.4 ... 89.4 89.6 89.8
    time       datetime64[ns] 8B 2025-09-09
Attributes:
    units:          m s-1
    valid_min:      -20.0
    valid_max:      20.0
    long_name:      Sea water y velocity
    short_name:     vomecrty
    standard_name:  sea_water_y_velocity
    cell_methods:   time: point
    grid_mapping:   latitude_longitude
    nomvar:         VVW
    stdscale:       1.0
    stdoffset:      0.0

In [18]:
ds.vomecrty[1,:,:,:]

IndexError: Index 1 is out of bounds for axis 0 with size 1

In [ ]:
ds.time

<xarray.DataArray 'time' (time: 11)> Size: 88B
array(['2025-09-09T00:00:00.000000000', '2025-09-10T00:00:00.000000000',
       '2025-09-11T00:00:00.000000000', '2025-09-12T00:00:00.000000000',
       '2025-09-13T00:00:00.000000000', '2025-09-14T00:00:00.000000000',
       '2025-09-15T00:00:00.000000000', '2025-09-16T00:00:00.000000000',
       '2025-09-17T00:00:00.000000000', '2025-09-18T00:00:00.000000000',
       '2025-09-09T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 88B 2025-09-09 2025-09-10 ... 2025-09-09
Attributes:
    long_name:      Validity time
    standard_name:  time
    axis:           T